# This code provide the demand file for FleetPy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Calculate the Need Proportion for Medical Need Rides

In [2]:
# Load medic visit need data
medic_visit_need = pd.read_csv('medic_visit_need.csv')
medic_visit_need

,tract_number,tract_name,Total Medical Visit,Total_med_rides_need
0,21001970100,"Adair County, Tract 970100",116.400718,17.591899
1,21001970200,"Adair County, Tract 970200",130.701292,18.882717
2,21001970300,"Adair County, Tract 970300",216.206251,37.235982
3,21001970401,"Adair County, Tract 970401",301.566243,53.424656
4,21001970402,"Adair County, Tract 970402",245.354618,34.877868
...,...,...,...,...
1110,21239050106,"Woodford County, Tract 050106",245.115946,39.115040
1111,21239050107,"Woodford County, Tract 050107",248.863732,39.954814
1112,21239050200,"Woodford County, Tract 050200",282.790542,41.632277
1113,21239050300,"Woodford County, Tract 050300",126.373998,17.301463


In [3]:
# Create a new column for need proportion
medic_visit_need['need_proportion'] = medic_visit_need['Total_med_rides_need']/medic_visit_need['Total Medical Visit']

In [4]:
medic_visit_need

,tract_number,tract_name,Total Medical Visit,Total_med_rides_need,need_proportion
0,21001970100,"Adair County, Tract 970100",116.400718,17.591899,0.151132
1,21001970200,"Adair County, Tract 970200",130.701292,18.882717,0.144472
2,21001970300,"Adair County, Tract 970300",216.206251,37.235982,0.172224
3,21001970401,"Adair County, Tract 970401",301.566243,53.424656,0.177157
4,21001970402,"Adair County, Tract 970402",245.354618,34.877868,0.142153
...,...,...,...,...,...
1110,21239050106,"Woodford County, Tract 050106",245.115946,39.115040,0.159578
1111,21239050107,"Woodford County, Tract 050107",248.863732,39.954814,0.160549
1112,21239050200,"Woodford County, Tract 050200",282.790542,41.632277,0.147219
1113,21239050300,"Woodford County, Tract 050300",126.373998,17.301463,0.136907


In [5]:
medic_visit_need['Total Medical Visit'].sum()

274623.8115576943

In [6]:
medic_visit_need['Total_med_rides_need'].sum()

42281.514099089174

In [7]:
medic_visit_need['need_proportion'].max()

0.20180487707224

In [8]:
# Save the sampled data to a new CSV file
medic_visit_need.to_csv("medic_visit_need_propotion.csv", index=False)

## Load the Gravity Matrix in Long Format DataFrame 

In [5]:
# Load the Medical visit and medical need ride data
gravity_matrix_long = pd.read_csv('gravity_matrix_long.csv')

gravity_matrix_long

,origin_geo_id,dest_geo_id,flow_count
0,21001970100,21001970100,0.000000
1,21001970100,21001970200,0.660443
2,21001970100,21001970300,0.078207
3,21001970100,21001970401,3.174550
4,21001970100,21001970402,5.640341
...,...,...,...
1243219,21239050400,21239050106,0.438467
1243220,21239050400,21239050107,0.841023
1243221,21239050400,21239050200,0.042113
1243222,21239050400,21239050300,0.091487


In [6]:
# Rename column appropriately
gravity_matrix_long.rename(columns={'flow_count': 'attracted_trips'}, inplace=True)

gravity_matrix_long

,origin_geo_id,dest_geo_id,attracted_trips
0,21001970100,21001970100,0.000000
1,21001970100,21001970200,0.660443
2,21001970100,21001970300,0.078207
3,21001970100,21001970401,3.174550
4,21001970100,21001970402,5.640341
...,...,...,...
1243219,21239050400,21239050106,0.438467
1243220,21239050400,21239050107,0.841023
1243221,21239050400,21239050200,0.042113
1243222,21239050400,21239050300,0.091487


In [7]:
# Set trips to UK healthcare destination (21067000801)
UK_destination = gravity_matrix_long[gravity_matrix_long['dest_geo_id'] == 21067000801]

UK_destination

,origin_geo_id,dest_geo_id,attracted_trips
270,21001970100,21067000801,3.283400
1385,21001970200,21067000801,0.782332
2500,21001970300,21067000801,3.873214
3615,21001970401,21067000801,2.667262
4730,21001970402,21067000801,1.725057
...,...,...,...
1237919,21239050106,21067000801,65.931136
1239034,21239050107,21067000801,57.088081
1240149,21239050200,21067000801,64.944824
1241264,21239050300,21067000801,24.980921


## Merge tract_name and need_proportion from medic_visit_need to UK_destination

In [8]:
# Merge need_proportion from medic_visit_need to UK_destination
UK_destination_merged = UK_destination.merge(
    medic_visit_need[['tract_number', 'tract_name', 'need_proportion']], 
    left_on='origin_geo_id', 
    right_on='tract_number', 
    how='left'
)

# Dropping the redundant tract_number column (since it is the same as origin_geo_id)
UK_destination_merged.drop(columns=['tract_number'], inplace=True)

# Display the first few rows of the merged dataset
UK_destination_merged


,origin_geo_id,dest_geo_id,attracted_trips,tract_name,need_proportion
0,21001970100,21067000801,3.283400,"Adair County, Tract 970100",0.151132
1,21001970200,21067000801,0.782332,"Adair County, Tract 970200",0.144472
2,21001970300,21067000801,3.873214,"Adair County, Tract 970300",0.172224
3,21001970401,21067000801,2.667262,"Adair County, Tract 970401",0.177157
4,21001970402,21067000801,1.725057,"Adair County, Tract 970402",0.142153
...,...,...,...,...,...
1110,21239050106,21067000801,65.931136,"Woodford County, Tract 050106",0.159578
1111,21239050107,21067000801,57.088081,"Woodford County, Tract 050107",0.160549
1112,21239050200,21067000801,64.944824,"Woodford County, Tract 050200",0.147219
1113,21239050300,21067000801,24.980921,"Woodford County, Tract 050300",0.136907


### Calculate number of people needing a medical ride from each census tract to UK Healthcare

In [9]:
# Calculate number of people needing a medical ride from each census tract to UK Healthcare

UK_destination_merged['need_med_ride'] = UK_destination_merged['attracted_trips'] * UK_destination_merged['need_proportion']

UK_destination_merged

,origin_geo_id,dest_geo_id,attracted_trips,tract_name,need_proportion,need_med_ride
0,21001970100,21067000801,3.283400,"Adair County, Tract 970100",0.151132,0.496228
1,21001970200,21067000801,0.782332,"Adair County, Tract 970200",0.144472,0.113025
2,21001970300,21067000801,3.873214,"Adair County, Tract 970300",0.172224,0.667062
3,21001970401,21067000801,2.667262,"Adair County, Tract 970401",0.177157,0.472525
4,21001970402,21067000801,1.725057,"Adair County, Tract 970402",0.142153,0.245222
...,...,...,...,...,...,...
1110,21239050106,21067000801,65.931136,"Woodford County, Tract 050106",0.159578,10.521139
1111,21239050107,21067000801,57.088081,"Woodford County, Tract 050107",0.160549,9.165432
1112,21239050200,21067000801,64.944824,"Woodford County, Tract 050200",0.147219,9.561143
1113,21239050300,21067000801,24.980921,"Woodford County, Tract 050300",0.136907,3.420058


In [10]:
UK_destination_merged.describe()

,origin_geo_id,dest_geo_id,attracted_trips,need_proportion,need_med_ride
count,1.115000e+03,1.115000e+03,1115.000000,1109.000000,1.109000e+03
mean,2.111067e+10,2.106700e+10,9.242970,0.153281,1.397711e+00
std,6.156300e+07,0.000000e+00,23.852237,0.011643,3.564826e+00
min,2.100197e+10,2.106700e+10,0.000000,0.130386,3.619483e-12
25%,2.106700e+10,2.106700e+10,0.138684,0.144555,2.285259e-02
50%,2.111101e+10,2.106700e+10,0.645565,0.151217,9.954607e-02
75%,2.115101e+10,2.106700e+10,4.483622,0.160019,6.861403e-01
max,2.123905e+10,2.106700e+10,192.718024,0.201805,2.702967e+01


In [11]:
UK_destination_merged['need_proportion'].mean()

0.15328051610824486

## Round the need_med_ride column 

In [12]:
import numpy as np
# Function to randomly round need med ride column
def random_round(value):
    mod = value % 1
    if np.random.rand() < mod:  # round up
        return np.ceil(value)
    else:  # round down
        return np.floor(value)

# Apply the function to the column
UK_destination_merged['rounded_need_med_ride'] = UK_destination_merged['need_med_ride'].apply(random_round)

UK_destination_merged

,origin_geo_id,dest_geo_id,attracted_trips,tract_name,need_proportion,need_med_ride,rounded_need_med_ride
0,21001970100,21067000801,3.283400,"Adair County, Tract 970100",0.151132,0.496228,0.0
1,21001970200,21067000801,0.782332,"Adair County, Tract 970200",0.144472,0.113025,0.0
2,21001970300,21067000801,3.873214,"Adair County, Tract 970300",0.172224,0.667062,0.0
3,21001970401,21067000801,2.667262,"Adair County, Tract 970401",0.177157,0.472525,1.0
4,21001970402,21067000801,1.725057,"Adair County, Tract 970402",0.142153,0.245222,1.0
...,...,...,...,...,...,...,...
1110,21239050106,21067000801,65.931136,"Woodford County, Tract 050106",0.159578,10.521139,11.0
1111,21239050107,21067000801,57.088081,"Woodford County, Tract 050107",0.160549,9.165432,10.0
1112,21239050200,21067000801,64.944824,"Woodford County, Tract 050200",0.147219,9.561143,10.0
1113,21239050300,21067000801,24.980921,"Woodford County, Tract 050300",0.136907,3.420058,3.0


In [73]:
save_file = UK_destination_merged.to_csv('UK_destination_merged.csv', index=False)

In [13]:
UK_destination_merged['rounded_need_med_ride'].sum()

1546.0

In [72]:
UK_destination_merged['attracted_trips'].sum()

10305.911743841152

In [14]:
# Rename column appropriately
UK_destination_merged.rename(columns={'rounded_need_med_ride': 'perfect_pax'}, inplace=True)

UK_destination_merged

,origin_geo_id,dest_geo_id,attracted_trips,tract_name,need_proportion,need_med_ride,perfect_pax
0,21001970100,21067000801,3.283400,"Adair County, Tract 970100",0.151132,0.496228,0.0
1,21001970200,21067000801,0.782332,"Adair County, Tract 970200",0.144472,0.113025,0.0
2,21001970300,21067000801,3.873214,"Adair County, Tract 970300",0.172224,0.667062,0.0
3,21001970401,21067000801,2.667262,"Adair County, Tract 970401",0.177157,0.472525,1.0
4,21001970402,21067000801,1.725057,"Adair County, Tract 970402",0.142153,0.245222,1.0
...,...,...,...,...,...,...,...
1110,21239050106,21067000801,65.931136,"Woodford County, Tract 050106",0.159578,10.521139,11.0
1111,21239050107,21067000801,57.088081,"Woodford County, Tract 050107",0.160549,9.165432,10.0
1112,21239050200,21067000801,64.944824,"Woodford County, Tract 050200",0.147219,9.561143,10.0
1113,21239050300,21067000801,24.980921,"Woodford County, Tract 050300",0.136907,3.420058,3.0


## Load KY Travel Time Data

In [15]:
# Load data
ky_travel_time = pd.read_csv('KY_travel_time_distance.csv')

ky_travel_time

,geoid_origin,origin_x,origin_y,geoid_dest,dest_x,dest_y,travel_distance,travel_time
0,2.123793e+10,-83.588704,37.738887,2.123793e+10,-83.588704,37.738887,0.000000,0.000000
1,2.123793e+10,-83.588704,37.738887,2.103193e+10,-86.731982,37.225166,212.131433,254.830000
2,2.123793e+10,-83.588704,37.738887,2.103193e+10,-86.723313,37.087762,218.796836,262.365000
3,2.123793e+10,-83.588704,37.738887,2.103193e+10,-86.517884,37.236586,206.800490,245.881667
4,2.123793e+10,-83.588704,37.738887,2.103193e+10,-86.848686,37.157033,221.002026,269.431667
...,...,...,...,...,...,...,...,...
1243219,2.119593e+10,-82.380711,37.517216,2.119593e+10,-82.486145,37.621373,23.598183,51.371667
1243220,2.119593e+10,-82.380711,37.517216,2.119593e+10,-82.470858,37.313760,38.993935,55.928333
1243221,2.119593e+10,-82.380711,37.517216,2.119593e+10,-82.562128,37.548172,20.724521,32.423333
1243222,2.119593e+10,-82.380711,37.517216,2.119593e+10,-82.192534,37.562992,22.869686,42.480000


In [16]:
# Strip spaces from column names to avoid hidden issues
ky_travel_time.columns = ky_travel_time.columns.str.strip()

In [17]:
# Convert geoid_origin and geoid_dest to whole numbers (integers)
ky_travel_time["geoid_origin"] = ky_travel_time["geoid_origin"].astype("int64")
ky_travel_time["geoid_dest"] = ky_travel_time["geoid_dest"].astype("int64")

In [18]:
ky_travel_time

,geoid_origin,origin_x,origin_y,geoid_dest,dest_x,dest_y,travel_distance,travel_time
0,21237930200,-83.588704,37.738887,21237930200,-83.588704,37.738887,0.000000,0.000000
1,21237930200,-83.588704,37.738887,21031930300,-86.731982,37.225166,212.131433,254.830000
2,21237930200,-83.588704,37.738887,21031930400,-86.723313,37.087762,218.796836,262.365000
3,21237930200,-83.588704,37.738887,21031930200,-86.517884,37.236586,206.800490,245.881667
4,21237930200,-83.588704,37.738887,21031930500,-86.848686,37.157033,221.002026,269.431667
...,...,...,...,...,...,...,...,...
1243219,21195931200,-82.380711,37.517216,21195930100,-82.486145,37.621373,23.598183,51.371667
1243220,21195931200,-82.380711,37.517216,21195931000,-82.470858,37.313760,38.993935,55.928333
1243221,21195931200,-82.380711,37.517216,21195930200,-82.562128,37.548172,20.724521,32.423333
1243222,21195931200,-82.380711,37.517216,21195931600,-82.192534,37.562992,22.869686,42.480000


## Set KY travel time to UK healthcare destination (21067000801)

In [19]:
# Set KY travel time to UK healthcare destination (21067000801)
UK_travel_time = ky_travel_time[ky_travel_time['geoid_dest'] == 21067000801]

UK_travel_time

,geoid_origin,origin_x,origin_y,geoid_dest,dest_x,dest_y,travel_distance,travel_time
955,21237930200,-83.588704,37.738887,21067000801,-84.503067,38.029765,65.408739,79.028333
2070,21031930300,-86.731982,37.225166,21067000801,-84.503067,38.029765,148.986106,185.936667
3185,21031930400,-86.723313,37.087762,21067000801,-84.503067,38.029765,155.651509,193.491667
4300,21031930200,-86.517884,37.236586,21067000801,-84.503067,38.029765,144.001703,177.725000
5415,21031930500,-86.848686,37.157033,21067000801,-84.503067,38.029765,157.856637,200.431667
...,...,...,...,...,...,...,...,...
1238605,21195931000,-82.470858,37.313760,21067000801,-84.503067,38.029765,164.232045,202.548333
1239720,21195930200,-82.562128,37.548172,21067000801,-84.503067,38.029765,139.721687,171.733333
1240835,21195931600,-82.192534,37.562992,21067000801,-84.503067,38.029765,168.855804,209.820000
1241950,21195930400,-82.481005,37.461693,21067000801,-84.503067,38.029765,147.997688,185.013333


## Merge need_proportion from medic_visit_need to UK_destination

In [20]:
# Merge need_proportion from medic_visit_need to UK_destination
UK_demand_time = UK_destination_merged.merge(
    UK_travel_time[['geoid_origin', 'travel_time', 'travel_distance']], 
    left_on='origin_geo_id', 
    right_on='geoid_origin', 
    how='left'
)

# Dropping the redundant tract_number column (since it is the same as origin_geo_id)
UK_demand_time.drop(columns=['geoid_origin'], inplace=True)

# Display the first few rows of the merged dataset
UK_demand_time


,origin_geo_id,dest_geo_id,attracted_trips,tract_name,need_proportion,need_med_ride,perfect_pax,travel_time,travel_distance
0,21001970100,21067000801,3.283400,"Adair County, Tract 970100",0.151132,0.496228,0.0,109.511667,76.315819
1,21001970200,21067000801,0.782332,"Adair County, Tract 970200",0.144472,0.113025,0.0,114.236667,96.260890
2,21001970300,21067000801,3.873214,"Adair County, Tract 970300",0.172224,0.667062,0.0,121.711667,84.799048
3,21001970401,21067000801,2.667262,"Adair County, Tract 970401",0.177157,0.472525,1.0,126.026667,98.933289
4,21001970402,21067000801,1.725057,"Adair County, Tract 970402",0.142153,0.245222,1.0,124.051667,102.547380
...,...,...,...,...,...,...,...,...,...
1110,21239050106,21067000801,65.931136,"Woodford County, Tract 050106",0.159578,10.521139,11.0,23.390000,11.451775
1111,21239050107,21067000801,57.088081,"Woodford County, Tract 050107",0.160549,9.165432,10.0,24.463333,16.278102
1112,21239050200,21067000801,64.944824,"Woodford County, Tract 050200",0.147219,9.561143,10.0,36.700000,20.590366
1113,21239050300,21067000801,24.980921,"Woodford County, Tract 050300",0.136907,3.420058,3.0,24.755000,17.000261


In [21]:
## Select relevant columns

UK_demand_time_df = UK_demand_time[['origin_geo_id', 'dest_geo_id', 'perfect_pax', 'travel_time']]

UK_demand_time_df

,origin_geo_id,dest_geo_id,perfect_pax,travel_time
0,21001970100,21067000801,0.0,109.511667
1,21001970200,21067000801,0.0,114.236667
2,21001970300,21067000801,0.0,121.711667
3,21001970401,21067000801,1.0,126.026667
4,21001970402,21067000801,1.0,124.051667
...,...,...,...,...
1110,21239050106,21067000801,11.0,23.390000
1111,21239050107,21067000801,10.0,24.463333
1112,21239050200,21067000801,10.0,36.700000
1113,21239050300,21067000801,3.0,24.755000


In [22]:
# Check rows with NaN values
nan_rows = UK_demand_time_df[UK_demand_time_df["perfect_pax"].isna()]
print(nan_rows)


      origin_geo_id  dest_geo_id  perfect_pax  travel_time
64      21015980100  21067000801          NaN    93.930000
124     21029980100  21067000801          NaN   128.073333
203     21047980100  21067000801          NaN   271.713333
678     21111980100  21067000801          NaN    96.973333
803     21143980100  21067000801          NaN   280.261667
1054    21221980200  21067000801          NaN   272.666667


In [23]:
# Store NaN values as zero and convert perfect_pax to integer

UK_demand_time_df["perfect_pax"] = pd.to_numeric(UK_demand_time_df["perfect_pax"], errors="coerce").fillna(0).astype("int64")

C:\Users\Musa\AppData\Local\Temp\ipykernel_15660\1209832893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK_demand_time_df["perfect_pax"] = pd.to_numeric(UK_demand_time_df["perfect_pax"], errors="coerce").fillna(0).astype("int64")


In [24]:
# Randomly round travel time
import numpy as np
# Function to randomly round travel time
def random_round(value):
    mod = value % 1
    if np.random.rand() < mod:  # round up
        return np.ceil(value)
    else:  # round down
        return np.floor(value)

# Apply the function to the column
UK_demand_time_df['travel time'] = UK_demand_time_df['travel_time'].apply(random_round)

C:\Users\Musa\AppData\Local\Temp\ipykernel_15660\1868262373.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK_demand_time_df['travel time'] = UK_demand_time_df['travel_time'].apply(random_round)


In [25]:
# Convert geoid_origin and geoid_dest to whole numbers (integers)
UK_demand_time_df["travel time"] = UK_demand_time_df["travel time"].astype("int64")

C:\Users\Musa\AppData\Local\Temp\ipykernel_15660\1370371533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK_demand_time_df["travel time"] = UK_demand_time_df["travel time"].astype("int64")


In [26]:
# Rename
UK_demand_time_df.rename(columns={'travel_time': 't_time'}, inplace=True)

C:\Users\Musa\AppData\Local\Temp\ipykernel_15660\3415290718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK_demand_time_df.rename(columns={'travel_time': 't_time'}, inplace=True)


In [27]:
UK_demand_time_df

,origin_geo_id,dest_geo_id,perfect_pax,t_time,travel time
0,21001970100,21067000801,0,109.511667,110
1,21001970200,21067000801,0,114.236667,115
2,21001970300,21067000801,0,121.711667,122
3,21001970401,21067000801,1,126.026667,126
4,21001970402,21067000801,1,124.051667,124
...,...,...,...,...,...
1110,21239050106,21067000801,11,23.390000,23
1111,21239050107,21067000801,10,24.463333,24
1112,21239050200,21067000801,10,36.700000,36
1113,21239050300,21067000801,3,24.755000,25


In [28]:
UK_demand_time_df['travel time'].mean()

121.51748878923767

## Develop the Ordinary Scheduling Approach

In [29]:
# Constants
HOSPITAL_OPEN_TIME = 28800  # 08:00 in seconds
HOSPITAL_CLOSE_TIME = 61200  # 17:00 in seconds
ARRIVE_EARLY_TIME = 900  # arrive to appointment 15 minutes early
APPOINTMENT_DURATION = 3600  # 1 hour (in seconds)
APPOINTMENT_INCREMENT = 900  # appointments occur in 15 minute increments

# List to store new records
expanded_data = []
request_id = 1

# Process each row in the dataset
for _, row in UK_demand_time_df.iterrows():
    origin = row["origin_geo_id"]
    destination = row["dest_geo_id"]
    perfect_pax = int(row["perfect_pax"]) # Ensure it's an integer
    travel_time = row["travel time"]
    
    for _ in range(perfect_pax):
        # Generate random appointment time within hospital hours
        appointment_time = np.random.randint(HOSPITAL_OPEN_TIME/APPOINTMENT_INCREMENT, HOSPITAL_CLOSE_TIME/APPOINTMENT_INCREMENT)
        appointment_time = appointment_time * APPOINTMENT_INCREMENT
        
        # Append go-trip
        expanded_data.append([appointment_time - (travel_time*60) - ARRIVE_EARLY_TIME, origin, destination, request_id, appointment_time])
        request_id += 1
        
        # Calculate return trip time
        return_rq_time = appointment_time + APPOINTMENT_DURATION
        
        # Append return-trip
        expanded_data.append([return_rq_time, destination, origin, request_id, appointment_time])
        request_id += 1

# Create new DataFrame
UKhealthcare_demand = pd.DataFrame(expanded_data, columns=["rq_time", "origin_geo_id", "dest_geo_id", "request_id", "appointment_time"])

In [30]:
UKhealthcare_demand

,rq_time,origin_geo_id,dest_geo_id,request_id,appointment_time
0,51840.0,2.100197e+10,2.106700e+10,1,60300
1,63900.0,2.106700e+10,2.100197e+10,2,60300
2,24960.0,2.100197e+10,2.106700e+10,3,33300
3,36900.0,2.106700e+10,2.100197e+10,4,33300
4,51240.0,2.100197e+10,2.106700e+10,5,60300
...,...,...,...,...,...
3087,51300.0,2.106700e+10,2.123905e+10,3088,47700
3088,34200.0,2.123905e+10,2.106700e+10,3089,36900
3089,40500.0,2.106700e+10,2.123905e+10,3090,36900
3090,46800.0,2.123905e+10,2.106700e+10,3091,49500


In [31]:
# Convert geoid_origin and geoid_dest to whole numbers (integers)
UKhealthcare_demand["rq_time"] = UKhealthcare_demand["rq_time"].astype("int64")
UKhealthcare_demand["origin_geo_id"] = UKhealthcare_demand["origin_geo_id"].astype("int64")
UKhealthcare_demand["dest_geo_id"] = UKhealthcare_demand["dest_geo_id"].astype("int64")

In [32]:
UKhealthcare_demand

,rq_time,origin_geo_id,dest_geo_id,request_id,appointment_time
0,51840,21001970401,21067000801,1,60300
1,63900,21067000801,21001970401,2,60300
2,24960,21001970402,21067000801,3,33300
3,36900,21067000801,21001970402,4,33300
4,51240,21001970500,21067000801,5,60300
...,...,...,...,...,...
3087,51300,21067000801,21239050400,3088,47700
3088,34200,21239050400,21067000801,3089,36900
3089,40500,21067000801,21239050400,3090,36900
3090,46800,21239050400,21067000801,3091,49500


## Use Nearest Neighbor from QGIS to Link Node Index with the Corresponding GEOID

In [33]:
# Load the nearest neighbor data saved from QGIS
nodes_neighbor = pd.read_csv('Ky_nearest_neighbor.csv')

In [34]:
nodes_neighbor = pd.DataFrame(nodes_neighbor)
nodes_neighbor

,fid,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,osm_id,is_stop_only,pos_x,pos_y,n,distance,feature_x,feature_y,nearest_x,nearest_y
0,1,21,237,930200,21237930200,9302.0,Census Tract 9302,G5020,S,248131879,...,168867810,False,-83.585382,37.744319,1,0,-83.585382,37.744319,-83.585382,37.744319
1,2,21,31,930300,21031930300,9303.0,Census Tract 9303,G5020,S,194557718,...,162492111,False,-86.713943,37.218780,1,0,-86.713943,37.218780,-86.713943,37.218780
2,3,21,31,930400,21031930400,9304.0,Census Tract 9304,G5020,S,218455330,...,162518877,False,-86.679174,37.161455,1,0,-86.679174,37.161455,-86.679174,37.161455
3,4,21,31,930200,21031930200,9302.0,Census Tract 9302,G5020,S,248312270,...,162503493,False,-86.431922,37.236734,1,0,-86.431922,37.236734,-86.431922,37.236734
4,5,21,31,930500,21031930500,9305.0,Census Tract 9305,G5020,S,208592384,...,162570711,False,-86.882836,37.156223,1,0,-86.882836,37.156223,-86.882836,37.156223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,1111,21,195,931000,21195931000,9310.0,Census Tract 9310,G5020,S,70345477,...,1293312684,False,-82.459456,37.361679,1,0,-82.459456,37.361679,-82.459456,37.361679
1111,1112,21,195,930200,21195930200,9302.0,Census Tract 9302,G5020,S,129254353,...,3239794745,False,-82.548504,37.510748,1,0,-82.548504,37.510748,-82.548504,37.510748
1112,1113,21,195,931600,21195931600,9316.0,Census Tract 9316,G5020,S,68228358,...,167784806,False,-82.200458,37.552682,1,0,-82.200458,37.552682,-82.200458,37.552682
1113,1114,21,195,930400,21195930400,9304.0,Census Tract 9304,G5020,S,48576134,...,637768897,False,-82.505627,37.442679,1,0,-82.505627,37.442679,-82.505627,37.442679


In [35]:
# Create a node lookup
node_lookup = dict(zip(nodes_neighbor['GEOID'], nodes_neighbor['node_index']))
node_lookup

{21237930200: 12190,
 21031930300: 7296,
 21031930400: 7314,
 21031930200: 7302,
 21031930500: 7331,
 21031930100: 7310,
 21111009300: 5649,
 21093001200: 7637,
 21111009600: 3551,
 21111009700: 5335,
 21111009800: 6530,
 21093000700: 7702,
 21093001500: 7568,
 21093001300: 7623,
 21093001700: 7580,
 21093001001: 7711,
 21093000902: 3380,
 21093000300: 6459,
 21093000800: 3371,
 21093001600: 7675,
 21093000901: 3373,
 21093001002: 7707,
 21137920300: 7898,
 21093000600: 9313,
 21093001100: 7704,
 21067003908: 10077,
 21067003909: 10838,
 21067003910: 10017,
 21067003911: 10085,
 21067002002: 2926,
 21111007000: 4252,
 21111007400: 6549,
 21111007601: 4297,
 21111007602: 3627,
 21137920400: 8903,
 21137920200: 7906,
 21035010100: 1778,
 21111007603: 5932,
 21067002001: 6832,
 21067003503: 2837,
 21067003504: 2978,
 21067000101: 3023,
 21067003912: 10088,
 21047201502: 1965,
 21047980100: 331,
 21047201302: 244,
 21093000202: 6201,
 21035010700: 140,
 21035010400: 1924,
 21035010200: 94,

In [36]:
node_lookup[21059000100]

576

## Apply node lookup to UKhealthcare_demand to link GEOID to node index

In [37]:
# Apply node lookup to UKhealthcare_demand to link the GEOID to the node index
UKhealthcare_demand['start'] = UKhealthcare_demand['origin_geo_id'].apply(lambda x : node_lookup[x])

In [38]:
UKhealthcare_demand['end'] = UKhealthcare_demand['dest_geo_id'].apply(lambda x : node_lookup[x])

In [39]:
UKhealthcare_demand

,rq_time,origin_geo_id,dest_geo_id,request_id,appointment_time,start,end
0,51840,21001970401,21067000801,1,60300,7118,2944
1,63900,21067000801,21001970401,2,60300,2944,7118
2,24960,21001970402,21067000801,3,33300,9262,2944
3,36900,21067000801,21001970402,4,33300,2944,9262
4,51240,21001970500,21067000801,5,60300,7065,2944
...,...,...,...,...,...,...,...
3087,51300,21067000801,21239050400,3088,47700,2944,5084
3088,34200,21239050400,21067000801,3089,36900,5084,2944
3089,40500,21067000801,21239050400,3090,36900,2944,5084
3090,46800,21239050400,21067000801,3091,49500,5084,2944


## Select relevant columns

In [40]:
# Select relevant columns
UKhealthcare_demand_data = UKhealthcare_demand[['rq_time', 'start', 'end', 'request_id']]

# Display the DataFrame
UKhealthcare_demand_data

,rq_time,start,end,request_id
0,51840,7118,2944,1
1,63900,2944,7118,2
2,24960,9262,2944,3
3,36900,2944,9262,4
4,51240,7065,2944,5
...,...,...,...,...
3087,51300,2944,5084,3088
3088,34200,5084,2944,3089
3089,40500,2944,5084,3090
3090,46800,5084,2944,3091


In [51]:
#Save file as csv
UKhealthcare_demand_data.to_csv('UKhealthcare_demand_data.csv', index=False)

In [52]:
UKhealthcare_demand_data['rq_time'].min()

14460

## Randomly Split Demand Request by 50% and 25%

In [53]:
# Load the Medical visit and medical need ride data
ukhealthcare_demand = pd.read_csv('UKhealthcare_demand_data.csv')

In [54]:
UKhealthcare_demand_data

,rq_time,start,end,request_id
0,51840,7118,2944,1
1,63900,2944,7118,2
2,24960,9262,2944,3
3,36900,2944,9262,4
4,51240,7065,2944,5
...,...,...,...,...
3087,51300,2944,5084,3088
3088,34200,5084,2944,3089
3089,40500,2944,5084,3090
3090,46800,5084,2944,3091


## 50% Data

In [55]:
UKhealthcare_demand_data

,rq_time,start,end,request_id
0,51840,7118,2944,1
1,63900,2944,7118,2
2,24960,9262,2944,3
3,36900,2944,9262,4
4,51240,7065,2944,5
...,...,...,...,...
3087,51300,2944,5084,3088
3088,34200,5084,2944,3089
3089,40500,2944,5084,3090
3090,46800,5084,2944,3091


In [56]:
# Randomly select 50% of the rows
ukhealthcare_demand_50 = UKhealthcare_demand_data.sample(frac=0.5, random_state=42)

In [57]:
ukhealthcare_demand_50

,rq_time,start,end,request_id
2743,39600,2944,5574,2744
1307,42300,2944,9988,1308
1602,34380,2950,2944,1603
2623,39600,2944,10584,2624
2031,38700,2944,6133,2032
...,...,...,...,...
2016,31440,3417,2944,2017
2420,30120,11863,2944,2421
2507,41400,2944,6628,2508
2238,40140,4078,2944,2239


In [58]:
# Save the sampled data to a new CSV file
ukhealthcare_demand_50.to_csv("ukhealthcare_demand_50.csv", index=False)

## 25% Data

In [68]:
# Randomly select 25% of the rows
ukhealthcare_demand_25 = ukhealthcare_demand.sample(frac=0.25, random_state=42)

In [69]:
ukhealthcare_demand_25

,rq_time,start,end,request_id,earliest_pickup_time,latest_pickup_time
679,36000,5442,2944,69,36000,39600
1637,63000,2944,2980,1038,63000,66600
568,33780,9849,2944,229,33780,37380
790,38100,3105,2944,1821,38100,41700
1767,57600,2944,3019,1498,57600,61200
...,...,...,...,...,...,...
2262,45900,2944,6465,1652,45900,49500
1552,54420,7003,2944,2099,54420,58020
123,24600,11219,2944,171,24600,28200
1536,54060,4091,2944,2277,54060,57660


In [70]:
# Save the sampled data to a new CSV file
ukhealthcare_demand_25.to_csv("ukhealthcare_demand_25.csv", index=False)

# Smart Demand Scheduling 

In [59]:
# Constants
HOSPITAL_OPEN_TIME = 28800  # 08:00 in seconds
HOSPITAL_CLOSE_TIME = 61200  # 17:00 in seconds
ARRIVE_EARLY_TIME = 900  # Arrive 15 minutes early
APPOINTMENT_DURATION = 3600  # 1 hour (in seconds)
APPOINTMENT_INCREMENT = 900  # Appointments in 15-minute increments

# List to store new records
expanded_data = []
request_id = 1

# Process each row in the dataset
for _, row in UK_demand_time_df.iterrows():
    origin = row["origin_geo_id"]
    destination = row["dest_geo_id"]
    perfect_pax = int(row["perfect_pax"])
    travel_time = row["travel time"]
    
    # Generate random appointment time within hospital hours
    appointment_time = np.random.randint(HOSPITAL_OPEN_TIME // APPOINTMENT_INCREMENT, HOSPITAL_CLOSE_TIME // APPOINTMENT_INCREMENT) * APPOINTMENT_INCREMENT

    for _ in range(perfect_pax):
        # Calculate go-trip request time
        rq_time = appointment_time - (travel_time * 60) - ARRIVE_EARLY_TIME
        
        # Append go-trip
        expanded_data.append([rq_time, origin, destination, request_id, appointment_time])
        request_id += 1
        
        # Calculate return trip time
        return_rq_time = appointment_time + APPOINTMENT_DURATION
        
        # Append return-trip
        expanded_data.append([return_rq_time, destination, origin, request_id, appointment_time])
        request_id += 1

# Create new DataFrame
UKhealthcare_demand_scheduling = pd.DataFrame(expanded_data, columns=["rq_time", "origin_geo_id", "dest_geo_id", "request_id", "appointment_time"])


In [61]:
UKhealthcare_demand_scheduling

,rq_time,origin_geo_id,dest_geo_id,request_id,appointment_time
0,23040.0,2.100197e+10,2.106700e+10,1,31500
1,35100.0,2.106700e+10,2.100197e+10,2,31500
2,45660.0,2.100197e+10,2.106700e+10,3,54000
3,57600.0,2.106700e+10,2.100197e+10,4,54000
4,20640.0,2.100197e+10,2.106700e+10,5,29700
...,...,...,...,...,...
3087,58500.0,2.106700e+10,2.123905e+10,3088,54900
3088,52200.0,2.123905e+10,2.106700e+10,3089,54900
3089,58500.0,2.106700e+10,2.123905e+10,3090,54900
3090,52200.0,2.123905e+10,2.106700e+10,3091,54900


In [62]:
# Apply node lookup to UKhealthcare_demand to link the GEOID to the node index
UKhealthcare_demand_scheduling['start'] = UKhealthcare_demand_scheduling['origin_geo_id'].apply(lambda x : node_lookup[x])

In [63]:
# Apply node lookup to UKhealthcare_demand to link the GEOID to the node index
UKhealthcare_demand_scheduling['end'] = UKhealthcare_demand_scheduling['dest_geo_id'].apply(lambda x : node_lookup[x])

In [64]:
UKhealthcare_demand_scheduling

,rq_time,origin_geo_id,dest_geo_id,request_id,appointment_time,start,end
0,23040.0,2.100197e+10,2.106700e+10,1,31500,7118,2944
1,35100.0,2.106700e+10,2.100197e+10,2,31500,2944,7118
2,45660.0,2.100197e+10,2.106700e+10,3,54000,9262,2944
3,57600.0,2.106700e+10,2.100197e+10,4,54000,2944,9262
4,20640.0,2.100197e+10,2.106700e+10,5,29700,7065,2944
...,...,...,...,...,...,...,...
3087,58500.0,2.106700e+10,2.123905e+10,3088,54900,2944,5084
3088,52200.0,2.123905e+10,2.106700e+10,3089,54900,5084,2944
3089,58500.0,2.106700e+10,2.123905e+10,3090,54900,2944,5084
3090,52200.0,2.123905e+10,2.106700e+10,3091,54900,5084,2944


In [65]:
# Select relevant columns
UKhealthcare_demand_scheduling = UKhealthcare_demand_scheduling[['rq_time', 'start', 'end', 'request_id']]

# Display the DataFrame
UKhealthcare_demand_scheduling

,rq_time,start,end,request_id
0,23040.0,7118,2944,1
1,35100.0,2944,7118,2
2,45660.0,9262,2944,3
3,57600.0,2944,9262,4
4,20640.0,7065,2944,5
...,...,...,...,...
3087,58500.0,2944,5084,3088
3088,52200.0,5084,2944,3089
3089,58500.0,2944,5084,3090
3090,52200.0,5084,2944,3091


In [66]:
# Save the sampled data to a new CSV file
UKhealthcare_demand_scheduling.to_csv("UKhealthcare_demand_scheduling.csv", index=False)

In [67]:
# Load the data
UKhealthcare_demand_scheduling = pd.read_csv('UKhealthcare_demand_scheduling.csv')

UKhealthcare_demand_scheduling

,rq_time,start,end,request_id
0,23040.0,7118,2944,1
1,35100.0,2944,7118,2
2,45660.0,9262,2944,3
3,57600.0,2944,9262,4
4,20640.0,7065,2944,5
...,...,...,...,...
3087,58500.0,2944,5084,3088
3088,52200.0,5084,2944,3089
3089,58500.0,2944,5084,3090
3090,52200.0,5084,2944,3091


## Randomly Split Demand Request by 50% and 25%

## 50% Data

In [68]:
# Randomly select 50% of the rows
ukhealthcare_demand_scheduling_50 = UKhealthcare_demand_scheduling.sample(frac=0.5, random_state=42)

ukhealthcare_demand_scheduling_50

,rq_time,start,end,request_id
2743,45000.0,2944,5574,2744
1307,60300.0,2944,9988,1308
1602,58680.0,2950,2944,1603
2623,54000.0,2944,10584,2624
2031,55800.0,2944,6133,2032
...,...,...,...,...
2016,51240.0,3417,2944,2017
2420,28320.0,11863,2944,2421
2507,37800.0,2944,6628,2508
2238,38340.0,4078,2944,2239


In [69]:
# Save the sampled data to a new CSV file
ukhealthcare_demand_scheduling_50.to_csv("ukhealthcare_demand_scheduling_50.csv", index=False)

## 25% Data

In [53]:
# Randomly select 25% of the rows
ukhealthcare_demand_scheduling_25 = UKhealthcare_demand_scheduling.sample(frac=0.25, random_state=42)

ukhealthcare_demand_scheduling_25

,rq_time,start,end,request_id,earliest_pickup_time,latest_pickup_time
1241,54900,2944,10742,1242,54900,58500
203,62100,2944,9874,204,62100,65700
1260,49380,10742,2944,1261,49380,52980
1626,32580,6465,2944,1627,32580,36180
1578,49740,2950,2944,1579,49740,53340
...,...,...,...,...,...,...
2932,21360,8663,2944,2933,21360,24960
2410,56520,11902,2944,2411,56520,60120
2337,48600,2944,11758,2338,48600,52200
962,35340,2998,2944,963,35340,38940


In [54]:
# Save the sampled data to a new CSV file
ukhealthcare_demand_scheduling_25.to_csv("ukhealthcare_demand_scheduling_25.csv", index=False)